## Colab 환경에서 하는 것을 권장합니다.

# 사전 준비

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
cd drive/MyDrive/TextMining_WordEmbedding

/content/drive/MyDrive/TextMining_WordEmbedding


In [3]:
#konlpy 설치 (mecab 제외). 3~40초 정도 소요
%%bash
apt-get update
apt-get install g++ openjdk-8-jdk python-dev python3-dev
pip3 install JPype1
pip3 install konlpy

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:2 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:6 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:10 http://security.ubuntu.com/ubuntu bionic-security/universe amd64 Packages [1,540 kB]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:12 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Get:13 http://archive.ubuntu.com/ubuntu bionic-updates/main amd64 Packages [3,390 kB]


In [4]:
# java 경로 설정
%env JAVA_HOME "/usr/lib/jvm/java-8-openjdk-amd64"

env: JAVA_HOME="/usr/lib/jvm/java-8-openjdk-amd64"


In [5]:
# 필요 라이브러리 불러오기
import os, json
from tqdm.notebook import tqdm

from konlpy.tag import Okt

from gensim.models import Word2Vec

import plotly
import plotly.graph_objs as go
import matplotlib.pyplot as plt
from sklearn.datasets import make_swiss_roll
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE

In [6]:
# data 불러오기
data_dir = os.path.join(os.getcwd(), 'data')

corpus = []
for i in tqdm(os.listdir(data_dir)):
    with open(os.path.join(data_dir, i), 'r') as f:
        json_data = json.load(f)
    corpus.append(json_data['sentences'])
corpus = sum(corpus, [])

  0%|          | 0/50 [00:00<?, ?it/s]

데이터를 확인하고 어떻게 전처리를 하고 토큰화를 해야 할 지 고민해보세요!

In [7]:
corpus[:50]

['12년 전 영국서 찾은 화석 분석… 초식공룡 이구아노돈 뇌조직 판명',
 '공룡의 뇌 화석(사진)이 세계 최초로 확인됐다',
 '데이비드 노먼 영국 케임브리지대 지구과학부 교수팀은 2004년 영국 서식스 주에서 발견된 화석을 분석한 결과 1억3300만 년 전에 살았던 것으로 추정되는 공룡의 뇌 조직인 것으로 밝혀졌다고 27일(현지 시간) 발표했다',
 '\u3000그동안 공룡의 뼈는 화석으로 많이 발견됐지만 뇌 부위는 부드럽고 부패가 빨리 진행돼 화석으로 남아 있는 경우가 없었다',
 '다른 화석으로 추정할 수밖에 없었던 공룡의 뇌 구조를 화석으로 직접 확인했다는 점에서 이번 발견은 의미를 가진다',
 '노먼 교수팀의 공룡 뇌가 화석으로 남아 있을 수 있었던 까닭은 공룡이 사망 직후 산소가 거의 없고 산성도가 높은 물에 머리가 잠겼기 때문이다',
 '일종의 ‘자연적 방부 처리’가 된 셈이다',
 '\u3000노먼 교수팀은 뇌 화석의 주인이 공룡 중에서도 ‘이구아노돈’일 것이라고 추정했다',
 '이구아노돈은 약 1억3300만 년 전, 중생대 백악기에 살았던 육상 공룡으로 몸길이 9m, 몸무게 3t 정도인 초식 공룡이다',
 '인공지능(AI) 알파고 개발자 데미스 하사비스 구글 딥마인드 최고경영자(CEO)가 이끄는 연구진이 신경망 컴퓨터와 외장 메모리만을 이용해 복잡한 지하철 노선에서 최단거리 찾기 같은 뇌의 추론 기능을 모방한 AI 기술을 개발했다',
 '구글 딥마인드는 신경망과 기존 컴퓨터의 장점을 결합해 아직은 초보적인 수준이지만 정확도가 높은 고차원적 추론이 가능한 하이브리드 기계학습 기술을 개발했다고 국제학술지 네이처 12일자에 발표했다',
 '컴퓨터 중앙처리장치(CPU)와 메모리로 이뤄진 일반 컴퓨터는 복잡한 대규모 자료를 처리하는데 유리하다',
 '최근 AI에서 주목받는 뇌 신경을 모방한 신경망 역시 기계학습을 통해 대규모 자료를 학습하고 이를 통해 특정한 패턴을 알아내는데 활용된다',
 '하지만 전문가들은 이 정도 기술은 뇌 일부 기능에 국한된다

# 1번

자신만의 CustomTokenization을 만들어보세요! 세션에서 말씀 드렸듯이 정답이 있는 것이 아닙니다. 

제가 세션에서 설명했던 것 처럼 pos를 이용하여 품사를 구하고 품사를 통해 stop words를 지정해서 제거해 줄 수 있고, 따로 stopw words를 지정해서 의미 없다고 생각하는 단어들을 제거해도 상관 없습니다!

물론 Okt말고 다른 Tokenizer를 사용해도 괜찮습니다. 

다양한 방법으로 전처리를 진행해보고 왜 그런 전처리와 tokenizing 방법을 사용했는지 설명해주세요!

[여러분의 설명]

주어진 corpus는 뉴스기사이기 때문에 감탄사, 외국어 및 기타 기호, 트위터 해쉬태그, ㅋㅋㅋ와 같은 구절, 숫자, 구두점, 트위터 아이디 등은 크게 중요하지 않을 것 같아서 불용어로 간주하여 제거해줌. 나아가 일반적으로 큰 의미를 가지지 않는 조사와 접미사도 제거함.

In [8]:
okt = Okt()

In [9]:
okt.tagset

{'Adjective': '형용사',
 'Adverb': '부사',
 'Alpha': '알파벳',
 'Conjunction': '접속사',
 'Determiner': '관형사',
 'Eomi': '어미',
 'Exclamation': '감탄사',
 'Foreign': '외국어, 한자 및 기타기호',
 'Hashtag': '트위터 해쉬태그',
 'Josa': '조사',
 'KoreanParticle': '(ex: ㅋㅋ)',
 'Noun': '명사',
 'Number': '숫자',
 'PreEomi': '선어말어미',
 'Punctuation': '구두점',
 'ScreenName': '트위터 아이디',
 'Suffix': '접미사',
 'Unknown': '미등록어',
 'Verb': '동사'}

In [10]:
# 자신만의 tokenization, preprocess 규칙을 만들고 그 규칙에 맞는 CustomTokenizer를 만들어보세요.
# 한 문장을 input으로 받아 token이 들어있는 list를 반환하는 함수를 구현해보세요.
def CustomTokenization(sentence):
  tokenized = []
  stopwords = ['Alpha', 'Exclamation', 'Foreign', 'Hashtag', 'Josa', 'KoreanParticle', 'Number', 'Punctuation', 'ScreenName', 'Suffix']
  for word, pos in okt.pos(sentence, norm=True, stem=True):
    if pos not in stopwords:
      tokenized.append(word)
  return tokenized

In [11]:
CustomTokenization('각각 실험에서 이 신경컴퓨터는 99.8% 정확도를 보인 것으로 나타났다')

['각각', '실험', '이', '신', '경', '컴퓨터', '정확도', '보이다', '것', '나타나다']

In [12]:
# corpus_tokenized 안에 CustomTokenization의 return값이 들어 있는 2중 list를 만들어 보세요
corpus_tokenized = []
for i in tqdm(corpus):
    corpus_tokenized.append(CustomTokenization(i))

  0%|          | 0/1020 [00:00<?, ?it/s]

In [13]:
corpus_tokenized

[['전', '영국', '찾다', '화석', '분석', '초식', '공룡', '이구아노돈', '뇌', '조직', '판명'],
 ['공룡', '뇌', '화석', '사진', '이', '세계', '최초', '확인', '돼다'],
 ['데이비드',
  '노먼',
  '영국',
  '케임브리지대',
  '지구',
  '과학부',
  '교수',
  '팀',
  '영국',
  '서식스',
  '주',
  '발견',
  '되다',
  '화석',
  '분석',
  '결과',
  '년',
  '전',
  '살다',
  '것',
  '추정',
  '되다',
  '공룡',
  '뇌',
  '조직',
  '것',
  '밝혀지다',
  '현지',
  '시간',
  '발표',
  '하다'],
 ['공룡',
  '뼈',
  '화석',
  '많이',
  '발견',
  '돼다',
  '뇌',
  '부위',
  '부드럽다',
  '부패',
  '빨리',
  '진행',
  '돼다',
  '화석',
  '남아',
  '있다',
  '경우',
  '없다'],
 ['다른',
  '화석',
  '추정',
  '하다',
  '수',
  '없다',
  '공룡',
  '뇌',
  '구조',
  '화석',
  '직접',
  '확인',
  '하다',
  '점',
  '이번',
  '발견',
  '의미',
  '가지다'],
 ['노먼',
  '교수',
  '팀',
  '공룡',
  '뇌',
  '화석',
  '남아',
  '있다',
  '수',
  '있다',
  '까닭',
  '공룡',
  '사망',
  '직후',
  '산소',
  '거의',
  '없다',
  '산',
  '성도',
  '높다',
  '물',
  '머리',
  '잠기다',
  '때문'],
 ['일종', '자연', '방부', '처리', '가다', '되다', '셈'],
 ['교수', '팀', '뇌', '화석', '주인', '공룡', '중', '이구아노돈', '일', '것', '추정', '하다'],
 ['이구아노돈',
  '약',
  '년',
  '전

# 2번

이제 word2vec 모델을 학습시켜 봅시다!

1. 사전학습 없이 모델 학습
2. 사전학습 된 모델을 사용하여 학습

두가지를 진행해보세요

hyperparameter는 본인이 하고 싶으신 것으로 하면 됩니다. 단, 사전학습 된 모델을 사용할 때는 size = 200으로 맞추어 주세요!

hyperparameter 설정 기준을 설명해주세요!

[여러분의 설명]

size는 통상적으로 100~300 사이로 설정한다는 내용을 인터넷에서 봤기 때문에 size는  200으로 설정해줌. CBOW와 skip-gram 중 skip-gram의 성능이 더 좋다고 배워서 sg=1으로 설정함으로써 skip-gram 선택함. learning rate와 비슷한 역할을 하는 alpha는 디폴트 값(0.025) 사용함.

In [14]:
# 사전학습 없이 모델 학습
# 모델 생성
model_no_pretrained = Word2Vec(size=200, min_count=3, window=3, sg=1, hs=1)

# 모델 vocabulary 추가
model_no_pretrained.build_vocab(corpus_tokenized)

total_examples = model_no_pretrained.corpus_count
print('현재 vocabulary size(V) :',len(model_no_pretrained.wv.vocab))

현재 vocabulary size(V) : 1298


In [15]:
# 모델 학습
model_no_pretrained.train(corpus_tokenized, total_examples = total_examples, epochs = 20)

(227785, 327140)

In [16]:
# 모델이 잘 학습 되었는지 자유롭게 확인해보세요!
# .wv.most_similar() 등 사용

model_no_pretrained.wv.most_similar('환경')

[('맞추다', 0.7757372856140137),
 ('받아들이다', 0.7484444379806519),
 ('토양', 0.743617057800293),
 ('마련', 0.730452299118042),
 ('나아가다', 0.7293407320976257),
 ('모니터링', 0.721765398979187),
 ('경찰', 0.687852144241333),
 ('증강현실', 0.6870677471160889),
 ('아바타', 0.6767610311508179),
 ('동작', 0.6767359972000122)]

In [17]:
model_no_pretrained.wv.most_similar('훈련')

[('연기', 0.8265705704689026),
 ('모으다', 0.785749077796936),
 ('불꽃놀이', 0.7674136161804199),
 ('연결', 0.7572697401046753),
 ('행동', 0.7481857538223267),
 ('돌아가다', 0.7451889514923096),
 ('이란', 0.7329275012016296),
 ('고리', 0.7316313982009888),
 ('내리다', 0.7305147647857666),
 ('거나', 0.7301895618438721)]

In [18]:
# 사전학습 된 모델을 사용하여 학습
#사전학습 된 모델 불러오기
pretrained = Word2Vec.load('ko.bin')

print('Pretrained model의 단어 수(V) :', len(pretrained.wv.vocab))
print('Pretrained model의 embedding vector의 size(N) :', len(pretrained.wv['강아지']))

# 모델 생성
model_pretrained = Word2Vec(size=200, min_count=6, window=6, sg=1, hs=0, alpha=0.05)

# 모델 vocabulary 추가
model_pretrained.build_vocab(corpus_tokenized)

total_examples = model_pretrained.corpus_count
print('현재 vocabulary size(V) :',len(model_pretrained.wv.vocab))

Pretrained model의 단어 수(V) : 30185
Pretrained model의 embedding vector의 size(N) : 200
현재 vocabulary size(V) : 649


In [19]:
#pretrained model의 학습되어 있는 vocabulary list 불러오기
model_pretrained.build_vocab([list(pretrained.wv.vocab.keys())], update = True)
print('현재 vocabulary size(V) :', len(model_pretrained.wv.vocab))

#embedding vector 까지 불러오기. lockf = 0.0 이면 불러온 벡터에 대해서는 재학습이 불가함. 1.0이면 재학습 가능.
model_pretrained.intersect_word2vec_format('ko.bin.gz', binary = False, lockf = 1.0)

현재 vocabulary size(V) : 649


In [20]:
# 모델 학습
model_pretrained.train(corpus_tokenized, total_examples = total_examples, epochs = 20)

(173539, 327140)

In [21]:
# 모델이 잘 학습 되었는지 자유롭게 확인해보세요!
# .wv.most_similar() 등 사용

model_pretrained.wv.most_similar('환경')

[('시스템', 0.5010480880737305),
 ('인프라', 0.4814378619194031),
 ('자연', 0.4752318859100342),
 ('특성', 0.4527429938316345),
 ('자율', 0.44000983238220215),
 ('상황', 0.42842960357666016),
 ('현실', 0.41963812708854675),
 ('개선', 0.41472315788269043),
 ('생체', 0.40842998027801514),
 ('능력', 0.4051457643508911)]

In [22]:
model_pretrained.wv.most_similar('훈련')

[('교육', 0.5767436027526855),
 ('비행', 0.4249570369720459),
 ('지원', 0.42068976163864136),
 ('학습', 0.401315301656723),
 ('의료', 0.3893663287162781),
 ('준비', 0.3804970383644104),
 ('치료', 0.36638399958610535),
 ('수행', 0.3626025915145874),
 ('시험', 0.36244115233421326),
 ('학생', 0.347001850605011)]

# 3번

사전 학습 없이 모델을 학습해도 성능이 괜찮나요?   
사전 학습 없이 학습한 모델과 사전 학습이 된 모델을 사용한 모델 간의 성능 차이가 있었나요? 차이가 있었다면 어떤 부분에서 차이가 나고 왜 그런 차이가 났는지 생각해보고 설명해주세요!

[여러분의 설명]

전반적으로 사전 학습이 된 모델의 성능이 더 좋음. 중심 단어를 넣었을 때 추출되는 문맥 단어가 사전 학습이 된 모델에서 더 개연성이 높음. 사전 학습 없이 만들어진 모델은 현재 주어진 데이터만을 기반으로 훈련된 반면 사전 학습이 된 모델은 훨씬 다양한 문맥에서 학습되었기 때문에 이 같은 결과가 나왔을 것으로 추정됨.

# 4번

이제 학습된 모델을 통해 나온 embedding 결과를 2차원, 3차원에 embedding 시켜 봅시다.    
사전학습된 데이터를 사용하면 너무 단어 수가 많아지기 때문에 우리가 처음에 가지고 있던 corpus_tokenized 데이터만 사용하겠습니다.   
PCA, t-SNE를 사용해서 시각화하고 그 두가지 방법의 결과를 비교해보세요!   

추가적으로 무언가 재미있는 시각화 결과 (ex. 이거랑 저거랑 비슷한 위치에 임베딩 됐어요!) 가 있다면 추가적으로 남겨주세요!

[여러분의 설명]

In [23]:
# 시각화할 단어 리스트 생성
unique_text = list(set(sum(corpus_tokenized, [])))
len(unique_text)

3589

In [48]:
# embedding 결과 가져오기
#사전 학습 되지 않은 모델 -> min_count=0으로 바꿈
model_no_pretrained = Word2Vec(size=200, min_count=0, window=3, sg=1, hs=1)
model_no_pretrained.build_vocab(corpus_tokenized)
total_examples = model_no_pretrained.corpus_count
model_no_pretrained.train(corpus_tokenized, total_examples = total_examples, epochs = 20)

#사전 학습된 모델 -> min_count=0으로 바꿈
pretrained = Word2Vec.load('ko.bin')
model_pretrained = Word2Vec(size=200, min_count=0, window=3, sg=1, hs=0, alpha=0.05)
model_pretrained.build_vocab(corpus_tokenized)
total_examples = model_pretrained.corpus_count
model_pretrained.build_vocab([list(pretrained.wv.vocab.keys())], update = True)
model_pretrained.intersect_word2vec_format('ko.bin.gz', binary = False, lockf = 1.0)
model_pretrained.train(corpus_tokenized, total_examples = total_examples, epochs = 20)

word, vector = [], []
for i in unique_text:
    word.append(i)
    vector.append(model_pretrained.wv[i])

In [49]:
pca = PCA(random_state=0).fit_transform(vector)
tsne = TSNE(n_components=2).fit_transform(vector)

/usr/local/lib/python3.7/dist-packages/sklearn/manifold/_t_sne.py:783: FutureWarning: The default initialization in TSNE will change from 'random' to 'pca' in 1.2.
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/manifold/_t_sne.py:793: FutureWarning: The default learning rate in TSNE will change from 200.0 to 'auto' in 1.2.
  FutureWarning,


In [51]:
# 시각화해보세요!
import plotly
import plotly.graph_objs as go
from sklearn.datasets import make_swiss_roll
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE

X, t = make_swiss_roll(n_samples=1000, noise=0)
color = (t- t.min()) / (t.max()-t.min())

data = go.Scatter3d(
    x=X[:,0],
    y=X[:,1],
    z=X[:,2],
    text = ['point #{}'.format(i) for i in range(X.shape[0])],
    mode='markers',
    marker=dict(
        size=3,
        color=color,
        colorscale='Jet',
        line=dict(
            width=0.0
        ),
        opacity=0.8
    )
)

layout = go.Layout(
    autosize=False,
    width=1000,
    height=1000,
    margin=go.Margin(
        l=50,
        r=50,
        b=100,
        t=100,
        pad=4
    ),
)

fig = go.Figure(data=[data], layout=layout)
fig.show()

/usr/local/lib/python3.7/dist-packages/plotly/graph_objs/_deprecations.py:410: DeprecationWarning: plotly.graph_objs.Margin is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.layout.Margin

  DeprecationWarning,


In [52]:
data = go.Scatter(
    x=pca[:,0],
    y=pca[:,1],
    text = ['point #{}'.format(i) for i in range(X.shape[0])],
    mode='markers',
    marker=dict(
        size=5,
        color=color,
        colorscale='Jet',
        line=dict(
            width=0.0
        ),
        opacity=0.8
    )
)

layout = go.Layout(
    autosize=False,
    width=1000,
    height=1000,
    margin=go.Margin(
        l=50,
        r=50,
        b=100,
        t=100,
        pad=4
    ),
)

fig = go.Figure(data=[data], layout=layout)
fig.show()

/usr/local/lib/python3.7/dist-packages/plotly/graph_objs/_deprecations.py:410: DeprecationWarning:

plotly.graph_objs.Margin is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.layout.Margin




In [53]:
data = go.Scatter(
    x=tsne[:,0],
    y=tsne[:,1],
    text = ['point #{}'.format(i) for i in range(X.shape[0])],
    mode='markers',
    marker=dict(
        size=5,
        color=color,
        colorscale='Jet',
        line=dict(
            width=0.0
        ),
        opacity=0.8
    )
)

layout = go.Layout(
    autosize=False,
    width=1000,
    height=1000,
    margin=go.Margin(
        l=50,
        r=50,
        b=100,
        t=100,
        pad=4
    ),
)

fig = go.Figure(data=[data], layout=layout)
fig.show()

# 5번

t-SNE에는 perplexity라는 hyperparameter가 존재했는데요. 5~50 사이의 값에서 robust 한 결과를 보인다고는 하지만 perplexity가 달라지면서 embedding 결과가 미묘하지만 크게 바뀌기도 합니다. perplexity와 n_iter 조합을 달리해보면서 perplexity의 변화가 embedding결과에 어떤 영향을 미치는지 살펴보세요!

In [54]:
# t-SNE
tsne = TSNE(perplexity=20, n_iter=500).fit_transform(vector)

# 시각화
data = go.Scatter(
    x=tsne[:,0],
    y=tsne[:,1],
    text = ['point #{}'.format(i) for i in range(X.shape[0])],
    mode='markers',
    marker=dict(
        size=5,
        color=color,
        colorscale='Jet',
        line=dict(
            width=0.0
        ),
        opacity=0.8
    )
)

layout = go.Layout(
    autosize=False,
    width=1000,
    height=1000,
    margin=go.Margin(
        l=50,
        r=50,
        b=100,
        t=100,
        pad=4
    ),
)

fig = go.Figure(data=[data], layout=layout)
fig.show()


/usr/local/lib/python3.7/dist-packages/sklearn/manifold/_t_sne.py:783: FutureWarning:

The default initialization in TSNE will change from 'random' to 'pca' in 1.2.

/usr/local/lib/python3.7/dist-packages/sklearn/manifold/_t_sne.py:793: FutureWarning:

The default learning rate in TSNE will change from 200.0 to 'auto' in 1.2.

/usr/local/lib/python3.7/dist-packages/plotly/graph_objs/_deprecations.py:410: DeprecationWarning:

plotly.graph_objs.Margin is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.layout.Margin




# 수고하셨습니다!